In [31]:
import torch
from torch import nn
from model_congater_adv import GateLayer
from model_congater_adv import ConGaterModel
from modeling import MDSTransformer
from main import get_query_encoder, get_loss_module
#from utils import load_model

In [2]:
cong = GateLayer(embed_size=765, squeeze_ratio=2)
cong

GateLayer(
  (linear): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=765, out_features=382, bias=True)
      (1): Tanh()
    )
    (1): Linear(in_features=382, out_features=765, bias=True)
  )
  (activation): Tsigmoid()
)

In [6]:
congmodel = ConGaterModel(model_name='google/bert_uncased_L-4_H-256_A-4', num_labels_task=2, num_labels_protected=2)
congmodel

Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ConGaterModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True

In [23]:
query_encoder_from = "sebastian-hofstaetter/distilbert-dot-tas_b-b256-msmarco"
d_model=None
num_heads=16
num_decoder_layers=1
dim_feedforward=966
dropout=0.1
activation="gelu"
normalization="LayerNorm"
doc_emb_dim=768,
scoring_mode="dot_product"
query_emb_aggregation="first"
loss_module = get_loss_module(args.loss_type, args)
aux_loss_module = None
# loss_module=loss_module,
# aux_loss_module=aux_loss_module,
# aux_loss_coeff=args.aux_loss_coeff,
# selfatten_mode=args.selfatten_mode,
no_decoder=True
# no_dec_crossatten=args.no_dec_crossatten,
# bias_regul_coeff=args.bias_regul_coeff,
# bias_regul_cutoff=args.bias_regul_cutoff
query_encoder = get_query_encoder(query_encoder_from=query_encoder_from, query_encoder_config=None)

2023-05-31 19:13:21,896 | root     - WARNING : Will initialize standard HuggingFace 'sebastian-hofstaetter/distilbert-dot-tas_b-b256-msmarco' as a query encoder!
2023-05-31 19:13:22,419 | root     - INFO : Query encoder loaded in 0.5215165615081787 s


In [30]:
query_encoder

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [24]:
irmodel = MDSTransformer(custom_encoder=query_encoder
                         , d_model=d_model
                         , num_heads=num_heads
                         , num_decoder_layers=num_decoder_layers
                         , dim_feedforward=dim_feedforward
                         , dropout=dropout
                         , activation=activation
                         , normalization=normalization
                         , doc_emb_dim=doc_emb_dim
                         , no_decoder= no_decoder)
irmodel

2023-05-31 19:13:23,790 | modeling - WARNING : No `d_model` provided. Will use (768,) dim. for transformer model dimension, to match expected document dimension!


TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of SymInts size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
